# T5 Translation Model

Summary:
* Replace slang text in reddit posts with de-slanged text
* Manually check replaced text to ensure posts make sense
* Train a T5 model on checked de-slanged text

In [1]:
!pip install -q transformers

In [2]:
!pip install simpletransformers

In [3]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline
from simpletransformers.t5 import T5Model, T5Args
import pandas as pd
import logging
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import json
import re
# Pull reddit data from reddit api
import requests
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_rows', 100)

## Pull Reddit Data

In [4]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('pigKA_TKnDkXcatEGcbo8g', 'nawGKK2MfPtC6vKz8TjaNEnmYfAggA')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'Katsuuu100',
        'password': 'Testing159753'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [5]:
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

# Pull from 5 classes. 5 classes = 5 subreddits.

my_list_of_dictionaries = []
total = 0
url_list_check = [f"https://oauth.reddit.com/r/wallstreetbets/new/",
          f"https://oauth.reddit.com/r/teenagers/new/",
          f"https://oauth.reddit.com/r/copypasta/new/",
          f"https://oauth.reddit.com/r/genz/new/",
          f"https://oauth.reddit.com/r/unpopularopinion/new/",
          f"https://oauth.reddit.com/r/frat/new/"
          ]

for i in range(len(url_list_check)):
    
  #print(url_list_check[i])
  res_check = requests.get(url_list_check[i],
                    headers=headers,
                    params={"limit": "1"})
  
  # print(json.dumps(res_check.json()["data"]["children"], indent=4))
  name = res_check.json()["data"]["children"][0]["data"]["name"]
  page_count = 25
  
  for j in range(4):

    url_list = [f"https://oauth.reddit.com/r/wallstreetbets/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/teenagers/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/copypasta/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/genz/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/unpopularopinion/new/?count={page_count}&after={name}",
            f"https://oauth.reddit.com/r/frat/new/?count={page_count}&after={name}"
            ]
    
    print("Page Count:", page_count)
    print("Name:", name)
    print("Url:", url_list[i])
    
    res = requests.get(url_list[i],
                    headers=headers)
                    # params={"limit": "100"})

    #print(json.dumps(res.json(), indent=4, sort_keys=False))

    reddit_dictionary = res.json()
    # print(json.dumps(reddit_dictionary["data"]["children"][0], indent=4, sort_keys=False))
    # print(len(reddit_dictionary["data"]["children"]))
    for k in range(len(reddit_dictionary["data"]["children"])):
      my_dictionary = {}
      my_dictionary["subreddit"] = reddit_dictionary["data"]["children"][k]["data"]["subreddit"]
      my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["selftext"]
      my_dictionary["title"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      my_dictionary["url"] = reddit_dictionary["data"]["children"][k]["data"]["url"]
      # print(reddit_dictionary["data"]["children"][k]["data"]["subreddit"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["selftext"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["url"])
      my_list_of_dictionaries.append(my_dictionary)
      total += 1
      name = reddit_dictionary["data"]["children"][k]["data"]["name"]
    # print(json.dumps(my_list_of_dictionaries, indent=4, sort_keys=False))
    
    page_count += 25

print("Total gathered:", total)

Page Count: 25
Name: t3_trszj0
Url: https://oauth.reddit.com/r/wallstreetbets/new/?count=25&after=t3_trszj0
Page Count: 50
Name: t3_trjoce
Url: https://oauth.reddit.com/r/wallstreetbets/new/?count=50&after=t3_trjoce
Page Count: 75
Name: t3_tr8kl5
Url: https://oauth.reddit.com/r/wallstreetbets/new/?count=75&after=t3_tr8kl5
Page Count: 100
Name: t3_tqsvw0
Url: https://oauth.reddit.com/r/wallstreetbets/new/?count=100&after=t3_tqsvw0
Page Count: 25
Name: t3_trt17o
Url: https://oauth.reddit.com/r/teenagers/new/?count=25&after=t3_trt17o
Page Count: 50
Name: t3_trstwt
Url: https://oauth.reddit.com/r/teenagers/new/?count=50&after=t3_trstwt
Page Count: 75
Name: t3_trsncg
Url: https://oauth.reddit.com/r/teenagers/new/?count=75&after=t3_trsncg
Page Count: 100
Name: t3_trsfwc
Url: https://oauth.reddit.com/r/teenagers/new/?count=100&after=t3_trsfwc
Page Count: 25
Name: t3_trsl0d
Url: https://oauth.reddit.com/r/copypasta/new/?count=25&after=t3_trsl0d
Page Count: 50
Name: t3_trhx0l
Url: https://oauth

In [6]:
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Convert list of dictionaries into pandas df
df = pd.DataFrame(my_list_of_dictionaries)
df.head()

,subreddit,text,title,url
0,wallstreetbets,"Is anyone aware of a list or service that tracks buyout, merger, takeover rumors? Perhaps as an alert or real-time list? Or at least some sort of summary that will collect such information into a centralized location? \n\nBasically I am looking for a trend of sorts to see and model when such rumors do occur and their generally frequency. Any location would be great to start my research.","Buyout, merger, takeover rumor notification service",https://www.reddit.com/r/wallstreetbets/comments/trsphf/buyout_merger_takeover_rumor_notification_service/
1,wallstreetbets,,The Other Apes,https://www.kapwing.com/videos/62426e25099433008dd84e20
2,wallstreetbets,,Update on TLRY position. Made it out with chump change. Lost slightly on the stock but made some on options.,https://www.reddit.com/gallery/trs814
3,wallstreetbets,,ᴛᴏᴍᴏʀʀᴏᴡ ᴡᴇ ʀɪᴅᴇ ᴏᴜᴛ ᴛᴏ ꜱᴍᴀꜱʜ ᴛʜᴇ ʀᴇꜱɪꜱᴛᴀɴᴄᴇ ᴀᴛ ᴛʜᴇ ᴄʀᴀᴄᴋ ᴏꜰ ᴅᴀᴡɴ,https://v.redd.it/1bgssne3ieq81
4,wallstreetbets,https://www.bloomberg.com/opinion/articles/2022-03-29/is-a-recession-coming-the-fed-has-made-it-inevitable?sref=uN6cur8D\n\nMay be time to poke the 🌈🐻s awake again.,"Former Head of NY Fed Bill Dudley: ""The Fed Has Made a U.S. Recession Inevitable""",https://www.reddit.com/r/wallstreetbets/comments/trs05w/former_head_of_ny_fed_bill_dudley_the_fed_has/


In [7]:
df['subreddit'].value_counts()

wallstreetbets      100
teenagers           100
copypasta           100
GenZ                100
unpopularopinion    100
Frat                100
Name: subreddit, dtype: int64

In [8]:
possible_labels = df.subreddit.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'wallstreetbets': 0,
 'teenagers': 1,
 'copypasta': 2,
 'GenZ': 3,
 'unpopularopinion': 4,
 'Frat': 5}

In [9]:
df['label'] = df.subreddit.replace(label_dict)
df.head()

,subreddit,text,title,url,label
0,wallstreetbets,"Is anyone aware of a list or service that tracks buyout, merger, takeover rumors? Perhaps as an alert or real-time list? Or at least some sort of summary that will collect such information into a centralized location? \n\nBasically I am looking for a trend of sorts to see and model when such rumors do occur and their generally frequency. Any location would be great to start my research.","Buyout, merger, takeover rumor notification service",https://www.reddit.com/r/wallstreetbets/comments/trsphf/buyout_merger_takeover_rumor_notification_service/,0
1,wallstreetbets,,The Other Apes,https://www.kapwing.com/videos/62426e25099433008dd84e20,0
2,wallstreetbets,,Update on TLRY position. Made it out with chump change. Lost slightly on the stock but made some on options.,https://www.reddit.com/gallery/trs814,0
3,wallstreetbets,,ᴛᴏᴍᴏʀʀᴏᴡ ᴡᴇ ʀɪᴅᴇ ᴏᴜᴛ ᴛᴏ ꜱᴍᴀꜱʜ ᴛʜᴇ ʀᴇꜱɪꜱᴛᴀɴᴄᴇ ᴀᴛ ᴛʜᴇ ᴄʀᴀᴄᴋ ᴏꜰ ᴅᴀᴡɴ,https://v.redd.it/1bgssne3ieq81,0
4,wallstreetbets,https://www.bloomberg.com/opinion/articles/2022-03-29/is-a-recession-coming-the-fed-has-made-it-inevitable?sref=uN6cur8D\n\nMay be time to poke the 🌈🐻s awake again.,"Former Head of NY Fed Bill Dudley: ""The Fed Has Made a U.S. Recession Inevitable""",https://www.reddit.com/r/wallstreetbets/comments/trs05w/former_head_of_ny_fed_bill_dudley_the_fed_has/,0


In [10]:
#combine title & text to make one column
df['title_and_text'] = df['title'] + ' ' +  df['text']
df['title_and_text'].head()

0    Buyout, merger, takeover rumor notification service Is anyone aware of a list or service that tracks buyout, merger, takeover rumors? Perhaps as an alert or real-time list? Or at least some sort of summary that will collect such information into a centralized location? \n\nBasically I am looking for a trend of sorts to see and model when such rumors do occur and their generally frequency. Any location would be great to start my research.
1                                                                                                                                                                                                                                                                                                                                                                                                                                              The Other Apes 
2                                                                                                         

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [12]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['subreddit', 'label', 'data_type']).count()

text  title  url  title_and_text
subreddit        label data_type                                  
Frat             5     train        85     85   85              85
                       val          15     15   15              15
GenZ             3     train        85     85   85              85
                       val          15     15   15              15
copypasta        2     train        85     85   85              85
                       val          15     15   15              15
teenagers        1     train        85     85   85              85
                       val          15     15   15              15
unpopularopinion 4     train        85     85   85              85
                       val          15     15   15              15
wallstreetbets   0     train        85     85   85              85
                       val          15     15   15              15

## Load Slangit Data

Slangit is a direct translation of slang data

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
slang = pd.read_csv('/content/gdrive/MyDrive/w266/final_project/slangit.csv')
slang.head(20)

,Slang Term,Meaning
0,*$,Starbucks
1,*$$,Starbucks
2,2,Two cents
3,0773H,Hello
4,10m,Ten man
5,10q,Thank you
6,10x,Thanks
7,1174,Meet in person at
8,121,One to one
9,1337,Leet


In [15]:
slangit_dict = slang.set_index('Slang Term').to_dict()
slangit_dict = slangit_dict['Meaning']

In [16]:
keys_values = slangit_dict.items()
slangit_dict = {str(key): str(value) for key, value in keys_values}

In [17]:
def slang_lookup(text, dictionary):
    
    pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in slangit_dict.keys()) + r')(?!\w)')
    result = pattern.sub(lambda x: dictionary[x.group()], text)

    return result

In [18]:
my_text = 'I watched the UNC game at a bar b/c YOLO, FTW'

print(slang_lookup(my_text, slangit_dict))

I watched the UNC game at a bar Be/See You only live once, For the win


## Apply Slangit regex replace to reddit data

In [19]:
df_train = df[df.index.isin(X_train)]
len(df_train)

510

In [20]:
df_train['title_and_text_deslanged'] = df_train['title_and_text'].apply(lambda x: slang_lookup(x, slangit_dict))
df_train['title_and_text_deslanged'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0    Buyout, merger, takeover rumor notification service Is anyone aware of a list or service that tracks buyout, merger, takeover rumors? Perhaps as an alert or real-time list? Or at least some sort of summary that will collect such information into a centralized location? \n\nBasically I am looking for a trend of sorts to see and model when such rumors do occur and their generally frequency. Any location would be great to start my research.
1                                                                                                                                                                                                                                                                                                                                                                                                                                              The Other Apes 
3                                                                                                         

In [21]:
df_train['title_and_text_deslanged'].head(30)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Buyout, merger, takeover rumor notification service Is anyone aware of a list or service that tracks buyout, merger, takeover rumors? Perhaps as an alert or real-time list? Or at least some sort of summary that will collect such information into a centralized location? \n\nBasically I am looking for a trend of sorts to see and model when such rumors do occur and their generally frequency. Any location would be great to start my rese

In [22]:
#df_train['same'] = df_train['title_and_text'].equals(df_train['title_and_text_deslanged'])
df_train['same'] = np.where(df_train['title_and_text'] == df_train['title_and_text_deslanged'] , True, False)
df_train['same'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0     True
1     True
3     True
4    False
5    False
Name: same, dtype: bool

In [23]:
df_train['same'].value_counts()

False    379
True     131
Name: same, dtype: int64

## Check regex deslang, correct examples where it is deslanged incorrectly

In [24]:
deslanged = df_train[['title_and_text', 'title_and_text_deslanged']][df_train['same'] == False]

In [25]:
deslanged[:100]

,title_and_text,title_and_text_deslanged
4,"Former Head of NY Fed Bill Dudley: ""The Fed Has Made a U.S. Recession Inevitable"" https://www.bloomberg.com/opinion/articles/2022-03-29/is-a-recession-coming-the-fed-has-made-it-inevitable?sref=uN6cur8D\n\nMay be time to poke the 🌈🐻s awake again.","Former Head of NY Fed Bill Dudley: ""The Fed Has Made a U.S. Recession Inevitable"" https://www.bloomberg.com/opinion/articles/2022-03-29/is-a-recession-coming-the-fed-has-made-it-inevitable?sref=uN6cur8D\n\nMay be time to poke the 🌈🐻Sarcasm awake again."
5,BBBY ~25K GAINZ...BEEN A GOOD YEAR,BBBY ~25K GAINZ...BEEN Assists GOOD YEAR
6,Picking up $GME is like investing in a dot com startup in the 90s. Fundamentally growing by the day.. 🐂ish AF still.,Picking Underpowered $GME is like investing in a dot com startup in the 90s. Fundamentally growing by the day.. 🐂ish As f*** still.
7,"I may be old school but... When I go to invest in the shares of a company, I think of myself as buying a part of that company. I ask myself ""would I want to own this company?"". Often I say ""hell yeah!"" who wouldn't want to own a company that is printing money? (like oil companies and banks and steel companies) and who wouldn't want to own a company that has truly disruptive technology that is going to change the world? (like Tesla and Amazon and Microsoft and Apple did) But then there are other times when I say ""not in a million years"" Why would I want to own a company that is bleeding cash and really has no prospects of ever turning a profit? One where the only way that I'm going to make money is to sell my shares to some sucker. When I find companies in the first category I buy their stock. When I find companies in the latter category I buy their puts. I am long in 60 different companies. Some of them are old school (value) some of them are new school (growth). I want t...","I may be old school but... When I go to invest in the shares of a company, I think of myself as buying a part of that company. I ask myself ""would I want to own this company?"". Often I say ""hell yeah!"" who wouldn't want to own a company that is printing money? (like oil companies and banks and steel companies) and who wouldn't want to own a company that has truly disruptive technology that is going to change the world? (like Tesla and Amazon and Microsoft and Apple did) But then there are other times when I say ""not in a million years"" Why would I want to own a company that is bleeding cash and really has Know prospects of ever turning a profit? One where the only way that I'Male going to make money is to sell my shares to some sucker. When I find companies in the first category I buy their stock. When I find companies in the latter category I buy their puts. I am long in 60 different companies. Some of them are old school (value) some of them are new school (growth). I w..."
8,Fomoing at 9:30 isn’t feeling too good right now mr. Stark…,Fomoing at Parent in room:30 isn’t feeling too good right now mr. Stark…
10,"what the FUCK would an ape do with PYPL I have a 186 cost basis. A few months ago every “expert” in the land was suckling at the PYPL teet. Then In a matter of a few weeks they all turned tail. In 2019 the full year EPS was 2.07 and the stock was around 120ish. Now they’re projecting high 4s EPS and the stock is at 120…wtf? So they lost eBay and the market treated it like it was gonna go BK because of that, who the fuck uses eBay anyway anymore. They gained Amazon with Venmo and the market barely gave 2 shits about that…AMAZON! \n\nI get their growth has slowed down due to less free allowance money from Powell out there, but that isn’t a PayPal story that’s a story every company almost. \n\nSo the stock has been destroyed thrown out to die. Finally starting to get some back but with a 186 cost basis im still down 30+ % \n\nWhat would an ape do here? Add more? Cut and run? Wait it out? \n\nIm not in TO deep but deeper than I wanna be so somewhat hesitant to add more","w

## T5 Translation

In [26]:
#model example from: https://simpletransformers.ai/docs/t5-data-formats/

import logging

import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_data = [
    ["binary classification", "Anakin was Luke's father" , "1"],
    ["binary classification", "Luke was a Sith Lord" , "0"],
    ["generate question", "Star Wars is an American epic space-opera media franchise created by George Lucas, which began with the eponymous 1977 film and quickly became a worldwide pop-culture phenomenon", "Who created the Star Wars franchise?"],
    ["generate question", "Anakin was Luke's father" , "Who was Luke's father?"],
]
train_df = pd.DataFrame(train_data)
train_df.columns = ["prefix", "input_text", "target_text"]

eval_data = [
    ["binary classification", "Leia was Luke's sister" , "1"],
    ["binary classification", "Han was a Sith Lord" , "0"],
    ["generate question", "In 2020, the Star Wars franchise's total value was estimated at US$70 billion, and it is currently the fifth-highest-grossing media franchise of all time.", "What is the total value of the Star Wars franchise?"],
    ["generate question", "Leia was Luke's sister" , "Who was Luke's sister?"],
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["prefix", "input_text", "target_text"]

model_args = T5Args()
model_args.num_train_epochs = 200
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

model = T5Model("t5", "t5-base", args=model_args, use_cuda=False)


def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])


model.train_model(train_df, eval_data=eval_df, matches=count_matches)

print(model.eval_model(eval_df, matches=count_matches))

ValueError: ignored

In [27]:
import logging

import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_data = df_train[['title_and_text', 'title_and_text_deslanged']]
train_data['prefix'] = 'translate'
train_df = train_data[['prefix', 'title_and_text', 'title_and_text_deslanged']]
train_df = train_df.rename(columns={'prefix': 'prefix', 'title_and_text': 'input_text', 'title_and_text_deslanged': 'target_text'})

eval_data = df[df.index.isin(X_val)]
eval_data['title_and_text_deslanged'] = eval_data['title_and_text'].apply(lambda x: slang_lookup(x, slangit_dict))
eval_data = eval_data[['title_and_text', 'title_and_text_deslanged']]
eval_data['prefix'] = 'translate'
eval_df = eval_data[['prefix', 'title_and_text', 'title_and_text_deslanged']]
eval_df = eval_df.rename(columns={'prefix': 'prefix', 'title_and_text': 'input_text', 'title_and_text_deslanged': 'target_text'})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
eval_df.head()

,prefix,input_text,target_text
2,translate,Update on TLRY position. Made it out with chump change. Lost slightly on the stock but made some on options.,Update on TLRY position. Made it out with chump change. Lost slightly on the stock but made some on options.
14,translate,"DWAC upcoming events DWAC is going live this weekend. Looks like Don Jr. joined TruthSocial yesterday March 28th, lots of ramping up going on and President Trump is closing in on 1M followers and I think he will start posting which will drive stock price higher if go-live is still by end of March on either Thursday or Friday this week.\n\nSo far, no comments from management except that they said ""Don't believe all the FUD articles out there, we're working around the clock to make the go-live happen by March 31st and onboard everyone"". \n\nPeople don't realize this is still in beta testing and hasn't gone into production yet. All of these fake articles about it being launched and president Trump is not posting etc, the truth is that authors are either too stupid or just flat out lying as it hasn't officially launched yet. Once it launches, you will see things change dramatically. \n\nWhat do you guys think? Think it will rally again given what's going on with memes?","DWAC upcoming events DWAC is going live this weekend. Looks like Don Jr. joined TruthSocial yesterday March 28th, lots of ramping Underpowered going on and President Trump is closing in on 1M followers and I think he will start posting which will drive stock price higher if go-live is still by end of March on either Thursday or Friday this week.\n\nSo far, Know comments from management except that they said ""Don't believe all the Fear, uncertainty, and doubt articles out there, Whatever'Rematch working around the clock to make the go-live happen by March 31st and onboard everyone"". \n\nPeople don't realize this is still in beta testing and hasn't gone into production yet. All of these fake articles about it being launched and president Trump is not posting etc, the truth is that authors are either too stupid or just flat out lying as it hasn't officially launched yet. Once it launches, you will see things change dramatically. \n\nWhat do you guys think? Think it will rally again gi..."
20,translate,"UVIX and SVIX a tale of two YOLOs There has been some discussion lately regarding VIX, the so called ""marker fear indicator"".\n\nThe smoothbrain summary is that VIX is based on SPX options and is inversely correlated with the market's performance. Market go down, VIX go up, and vice versa. Whenever a black swan market catasrophe occurs, VIX will shoot up, such as in the wake of the 2020 COVID crash. \n\nTwo new securities become tradable tomorrow:\nUVIX, a 2X leveraged ETF of VIX, and SVIX a 1X inverse (short) ETF of VIX. \n\nCurrently UVXY is tradable which is a pussy baby bitch boy version of UVIX, only 1.5x leveraged. But I'm such a degenerate gambler that 1.5x leverage doesn't get me hard anymore, so I'm glad they're coming out with the 2x. \n\nNeat fact, these securities equivilents were discontinued in 2018 due to an event known as ""volmageddon"" where volatility got so crazy that it almost wiped the funds out. And now they're back. What makes them think this time will be any ...","UVIX and SVIX a tale of two YOLOs There has been some discussion lately regarding VIX, the so called ""marker fear indicator"".\n\nThe smoothbrain summary is that VIX is based on SPX options and is inversely correlated with the market'Sarcasm performance. Market go down, VIX go Underpowered, and vice versa. Whenever a black swan market catasrophe occurs, VIX will shoot Underpowered, such as in the wake of the 2020 COVID crash. \n\nTwo new securities become tradable tomorrow:\nUVIX, a 2X leveraged Exchange traded fund of VIX, and SVIX a 1X inverse (short) Exchange traded fund of VIX. \n\nCurrently UVXY is tradable which is a pussy baby bitch boy version of UVIX, only 1.5x leveraged. But I'Male such a degenerat

In [29]:
model_args = T5Args()
model_args.num_train_epochs = 10
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True
torch.cuda.memory_summary(device=None, abbreviated=False)
model_args.per_gpu_train_batch_size = 128

model = T5Model("t5", "t5-base", args=model_args, use_cuda=True)


In [30]:
def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])

In [31]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [32]:
torch.cuda.empty_cache()

In [33]:
model.train_model(train_df, eval_data=eval_df, matches=count_matches)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/510 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-base_cached_128510
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/64 [00:00<?, ?it/s]

...

Generating outputs:   0%|          | 0/12 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/90 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 0.22860282162825266, 'matches': 3}
INFO:simpletransformers.t5.t5_model: Training of t5-base model complete. Saved to outputs/.


['Update on TLRY position. Made it out with chump change. Lost slightly on the stock but made some on options. ', 'DWAC upcoming events DWAC is going live this weekend. Looks like Don Jr. joined TruthSocial yesterday March 28th, lots of ramping Underpowered going on and President Trump is closing in on 1M followers and I think he will start posting which will drive stock price higher if go-live is still by end of March on either Thursday or Friday this week.\n\nSo far, Know comments from management except that they said "Don\'t believe all the Fear, uncertainty, and doubt articles out there, Whatever\'Rematch working around the clock to make the go-live happen by March 31st and onboard everyone". \n\nPeople don\'t realize this is still in beta testing and hasn\'t gone into production yet. All of these fake articles about it being launched and president Trump is not posting etc, the truth is that authors are either too stupid or just flat out lying as it hasn\'t officially launched yet.

(640,
 {'global_step': [64, 128, 192, 256, 320, 384, 448, 512, 576, 640],
  'eval_loss': [0.197124062726895,
   0.16987959202378988,
   0.17133385501801968,
   0.19087559605638185,
   0.1919170261050264,
   0.2217534463852644,
   0.21112394829591116,
   0.2137157010535399,
   0.21528656035661697,
   0.22860282162825266],
  'train_loss': [0.15156328678131104,
   0.09466961026191711,
   0.04787181317806244,
   0.030529744923114777,
   0.11311999708414078,
   0.0033521843142807484,
   0.01663360185921192,
   0.015041339211165905,
   0.022506095468997955,
   0.031670451164245605],
  'matches': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]})

In [34]:
my_t5 = model.eval_model(eval_df, matches=count_matches)
print(my_t5)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/90 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-base_cached_12890


Running Evaluation:   0%|          | 0/12 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/90 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 0.22860282162825266, 'matches': 3}


['Update on TLRY position. Made it out with chump change. Lost slightly on the stock but made some on options. ', 'DWAC upcoming events DWAC is going live this weekend. Looks like Don Jr. joined TruthSocial yesterday March 28th, lots of ramping Underpowered going on and President Trump is closing in on 1M followers and I think he will start posting which will drive stock price higher if go-live is still by end of March on either Thursday or Friday this week.\n\nSo far, Know comments from management except that they said "Don\'t believe all the Fear, uncertainty, and doubt articles out there, Whatever\'Rematch working around the clock to make the go-live happen by March 31st and onboard everyone". \n\nPeople don\'t realize this is still in beta testing and hasn\'t gone into production yet. All of these fake articles about it being launched and president Trump is not posting etc, the truth is that authors are either too stupid or just flat out lying as it hasn\'t officially launched yet.

In [36]:
print(my_t5)

{'eval_loss': 0.22860282162825266, 'matches': 3}


In [39]:
#get predictions included in df
preds = model.predict(list(eval_df['input_text']))

Generating outputs:   0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/90 [00:00<?, ?it/s]

In [40]:
print(preds)

['Make it out with chump change. Lost slightly on the stock but made some on options', 'So far, Know comments from management except that they said "Don\'t believe all the F', 'Whenever a black swan market catasrophe occurs, VIX will', 'Twitter user doesn’t know why this went red today', 'Pain', 'YouTube channel on how to become a millionaire', 'I was watching this as it was happening. LFG reached $448,950 seconds before', 'Give me A stock split on gamma, delta, and IV Tl;d', 'Jim Cramer running his Hedge Fund back in 1997, a year later (1998)', 'Jim Cramer declares the bear market is over Sauce: [https://www.', 'What I think when I see your gain porn', 'My little yolo', 'The dragon coils Underpowered drawing power from its neck, its back and its crack until it', "Don't you dare disable the buy button again, Kenny!", "Hi everyone, just a quick post about home depot again. I've been following it", 'AI generated image of love', 'Interception cant remember if something happened in 2020 or

In [41]:
eval_df['t5_prediction'] = preds
eval_df.head()

,prefix,input_text,target_text,t5_prediction
2,translate,Update on TLRY position. Made it out with chump change. Lost slightly on the stock but made some on options.,Update on TLRY position. Made it out with chump change. Lost slightly on the stock but made some on options.,Make it out with chump change. Lost slightly on the stock but made some on options
14,translate,"DWAC upcoming events DWAC is going live this weekend. Looks like Don Jr. joined TruthSocial yesterday March 28th, lots of ramping up going on and President Trump is closing in on 1M followers and I think he will start posting which will drive stock price higher if go-live is still by end of March on either Thursday or Friday this week.\n\nSo far, no comments from management except that they said ""Don't believe all the FUD articles out there, we're working around the clock to make the go-live happen by March 31st and onboard everyone"". \n\nPeople don't realize this is still in beta testing and hasn't gone into production yet. All of these fake articles about it being launched and president Trump is not posting etc, the truth is that authors are either too stupid or just flat out lying as it hasn't officially launched yet. Once it launches, you will see things change dramatically. \n\nWhat do you guys think? Think it will rally again given what's going on with memes?","DWAC upcoming events DWAC is going live this weekend. Looks like Don Jr. joined TruthSocial yesterday March 28th, lots of ramping Underpowered going on and President Trump is closing in on 1M followers and I think he will start posting which will drive stock price higher if go-live is still by end of March on either Thursday or Friday this week.\n\nSo far, Know comments from management except that they said ""Don't believe all the Fear, uncertainty, and doubt articles out there, Whatever'Rematch working around the clock to make the go-live happen by March 31st and onboard everyone"". \n\nPeople don't realize this is still in beta testing and hasn't gone into production yet. All of these fake articles about it being launched and president Trump is not posting etc, the truth is that authors are either too stupid or just flat out lying as it hasn't officially launched yet. Once it launches, you will see things change dramatically. \n\nWhat do you guys think? Think it will rally again gi...","So far, Know comments from management except that they said ""Don't believe all the F"
20,translate,"UVIX and SVIX a tale of two YOLOs There has been some discussion lately regarding VIX, the so called ""marker fear indicator"".\n\nThe smoothbrain summary is that VIX is based on SPX options and is inversely correlated with the market's performance. Market go down, VIX go up, and vice versa. Whenever a black swan market catasrophe occurs, VIX will shoot up, such as in the wake of the 2020 COVID crash. \n\nTwo new securities become tradable tomorrow:\nUVIX, a 2X leveraged ETF of VIX, and SVIX a 1X inverse (short) ETF of VIX. \n\nCurrently UVXY is tradable which is a pussy baby bitch boy version of UVIX, only 1.5x leveraged. But I'm such a degenerate gambler that 1.5x leverage doesn't get me hard anymore, so I'm glad they're coming out with the 2x. \n\nNeat fact, these securities equivilents were discontinued in 2018 due to an event known as ""volmageddon"" where volatility got so crazy that it almost wiped the funds out. And now they're back. What makes them think this time will be any ...","UVIX and SVIX a tale of two YOLOs There has been some discussion lately regarding VIX, the so called ""marker fear indicator"".\n\nThe smoothbrain summary is that VIX is based on SPX options and is inversely correlated with the market'Sarcasm performance. Market go down, VIX go Underpowered, and vice versa. Whenever a black swan market catasrophe occurs, VIX will shoot Underpowered, such as in the wake of the 2020 COVID crash. \n\nTwo new securities become tradable tomorrow:\nUVIX, a 2X leveraged Exchange traded fund of VIX, an